# Preliminaries 
Choose a dataset and set simulation parameters

In [12]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
# from classes.params import simul_param, fl_param

# load a tabular dataset for multiclass classification (example with scikitlearn datasets)
num_classes = 4
X, y = datasets.make_classification(n_samples=1000, n_features=10, n_informative=5, n_redundant=5, n_classes=4)
x_train, x_valid,  y_train, y_valid = train_test_split(X, y, test_size=0.4, random_state=42)
    

Sets the number of clients and the number of trees (xgboost) per client

In [13]:
num_clients = 5  # K
trees_client = 10  # M

# Centralized performance
Data are fused on the server, this is the classical distributed xboost, privacy critical


In [14]:
import joblib
from utils import accuracy
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import xgboost as xgb
import numpy as np

# xgboost parameters (example)
hyperparams = {
    "objective": "multi:softmax",
    # Same number of trees as in the decentralized case
    "n_estimators": num_clients * trees_client,
    "max_depth": 5,
    "learning_rate": 0.1,
    "base_score": 0.5,
    "random_state": 34,
}

reg = xgb.XGBClassifier(**hyperparams)
reg.fit(x_train, y_train)
y_pred = reg.predict(x_valid)
accuracy_s = accuracy_score(y_valid, y_pred)

print(f"Accuracy: {accuracy_s:.2f}") 

cm = confusion_matrix(y_valid, y_pred)
# save and store the centralized model
checkpointpath1 = 'xgb_models/XGB_centralized_model.h5'
joblib.dump(reg, checkpointpath1, compress=0)


Accuracy: 0.78


['xgb_models/XGB_centralized_model.h5']

# Isolated case (no federation) 
Training of local xgboost models (base models of the ensemble)

Code below implements iid split (can be extened with sample/label/feature imbalance), saves training, validation data in data/client_i and server parameters in server folders


In [15]:
from classes.Datasets.dataset_client import Dataset
from classes.Datasets.data_partitioner import split_iid_sim
import os, json
# or run python -m classes.Datasets.data_generator.py to get a data distribution
samples = 100
niid_type = 'iid'
alpha = 1

print('Splitting IID')

# split the training dataset and create folders in data/client_#i/train
split_iid_sim(x_train, y_train, samples, num_clients, type='train')

# split the validation dataset and create folders in data/client_#i/valid
split_iid_sim(x_valid, y_valid, samples, num_clients, type='valid')

x_train_clients = []
y_train_clients = []
x_valid_clients = []
y_valid_clients = []

# create train and valid datasets for all clients
for k in range(num_clients):
    handle = Dataset(k) # get an handle to training dataset of client k
    x_train_clients.append(handle.x_train_local)
    y_train_clients.append(handle.y_train_local)
    x_valid_clients.append(handle.x_valid)
    y_valid_clients.append(handle.y_valid)

datasets = tuple(zip(x_train_clients, y_train_clients))


Splitting IID
Client 0 | Samples 100
Client 1 | Samples 100
Client 2 | Samples 100
Client 3 | Samples 100
Client 4 | Samples 100
Saved train data
Client 0 | Samples 80
Client 1 | Samples 80
Client 2 | Samples 80
Client 3 | Samples 80
Client 4 | Samples 80
Saved valid data


Train the xboost tree models locally. Decision tree models are the ensemble model (base models) for fedxbgoostllr. Save the ensembles and evaluate them separately (no federation)

In [16]:
# Hyperparameters for each of the clients
hyperparams = {
    "objective": "multi:softmax",
    "n_estimators": trees_client,
    "max_depth": 5,
    "learning_rate": 0.1,
    "base_score": 0.5,  # np.mean(y_train)
    "random_state": 34,
}

errors_clients = []

for c, (x_train, y_train) in enumerate(
        datasets
):  # extract the dataset for the current client
    reg = xgb.XGBClassifier(**hyperparams) # train the classifier
    reg.fit(x_train, y_train)
    # save model
    checkpointpath = 'xgb_models/XGB_client_model_{}.h5'.format(c)
    joblib.dump(reg, checkpointpath, compress=0)
    
    # full performance tests (accuracy and confusion matrix)
    y_pred = reg.predict(x_valid)
    error = accuracy_score(y_valid, y_pred)
    cm = confusion_matrix(y_valid, y_pred)
    print(f"xgboost classifier local model accuracy, (Client {c}): {100*error :.5f}%")
    errors_clients.append(error)



xgboost classifier local model accuracy, (Client 0): 58.50000%
xgboost classifier local model accuracy, (Client 1): 59.25000%
xgboost classifier local model accuracy, (Client 2): 64.25000%
xgboost classifier local model accuracy, (Client 3): 60.75000%
xgboost classifier local model accuracy, (Client 4): 65.25000%


# Federated XGBoost 
The global model is a 1D-CNN type with specific filter sizes. The global model acts as an "ensemble model"

The pipeline is the following (XGB trees outputs-> 1D-CNN -> predictions)

In the following the local xgboost models are now re-trained to solve a one vs all classification problem. The outputs of the local xgboost models are treated as inputs to the 1D-CNN model.

In [17]:
################################# INDIVIDUAL CLIENTS XGBOOST REGRESSION MODEL TRAINING)
from sklearn.multiclass import OneVsRestClassifier

# Hyperparameters for each of the clients
hyperparams = {
    # "objective": "reg:squarederror",
    "objective": "binary:logistic",
    "n_estimators": trees_client,
    "max_depth": 5,
    "learning_rate": 0.1,
    "base_score": 0.5,  # np.mean(y_train)
    "random_state": 34,
}

for c, (x_train, y_train) in enumerate(
        datasets
):  # extract the dataset for the current client
    reg = OneVsRestClassifier(xgb.XGBClassifier(**hyperparams)) # regression model training
    reg.fit(x_train, y_train)
    # save model
    checkpointpath = 'xgb_models/XGB_client_model_fed_{}.h5'.format(c)
    joblib.dump(reg, checkpointpath, compress=0)
  
# test of the one vs all local model (compared with the previous case)
    y_pred = reg.predict(x_valid)
    error = accuracy_score(y_valid, y_pred)
    cm = confusion_matrix(y_valid, y_pred)
    print(f"One vs all local model accuracy, (Client {c}): {100*error :.5f}%")





One vs all local model accuracy, (Client 0): 56.25000%
One vs all local model accuracy, (Client 1): 55.50000%
One vs all local model accuracy, (Client 2): 66.75000%
One vs all local model accuracy, (Client 3): 61.00000%
One vs all local model accuracy, (Client 4): 62.25000%


Create a new "dataset" input to 1D-CNN which consists of the XGB trees regression models previously trained (locally):

NOTE: During initialization, all xgboost models (of all clients) must be shared with all clients before starting the FL process. MQTT can be used for this (but also other methods apply). In the following xgboost base models are loaded from a shared folder. 

In [18]:
from utils import get_trees_predictions_xgb

# load all xgboost models and prepare the data
XGB_models = []
for c in range(num_clients):
    checkpointpath1 = 'xgb_models/XGB_client_model_fed_{}.h5'.format(c)
    xgb = joblib.load(checkpointpath1)
    classifiers = xgb.estimators_
    for q in range(num_classes):
        XGB_models.append(classifiers [q])

# prepare the new dataset for training
objective = "binary"
x_xgb_trees_out = []
y_xgb_trees_out = []
for c, (x_train, y_train) in enumerate(datasets):  # for each client
    print("Converting the data of client", c, 100 * "-")
    # XGB trees outputs (for all XGBoost trees!) corresponding to training data of client c
    # NOTE: numclasses is needed to clip the inputs
    x_xgb_trees_out.append(get_trees_predictions_xgb(x_train, objective, *XGB_models, numclasses=num_classes)) 
    categorical_labels = np.squeeze(np.eye(num_classes)[y_train.reshape(-1)])
    y_xgb_trees_out.append(categorical_labels) # true labels now categorical

datasets_out = tuple(zip(x_xgb_trees_out, y_xgb_trees_out)) # dataset_out is the new federated dataset input to 1D-CNN (XGB trees output-> 1D-CNN -> accuracy)

# Validation data

xgb_valid_out = get_trees_predictions_xgb(x_valid, objective, *XGB_models, numclasses=num_classes) # XGB trees outputs corresponding to validation data: to simplify the reasoning, we apply same validation set for all (other options are also feasible)


Converting the data of client 0 ----------------------------------------------------------------------------------------------------
Converting the data of client 1 ----------------------------------------------------------------------------------------------------
Converting the data of client 2 ----------------------------------------------------------------------------------------------------
Converting the data of client 3 ----------------------------------------------------------------------------------------------------
Converting the data of client 4 ----------------------------------------------------------------------------------------------------


# FedXGBooost aggregator 
initialize the global model (or ensemble model)

In [19]:
from models import CNN_mc

filters = 32 # convolutional filters (16, 32 ok, >32 too large, depends on tree structures) TO BE OPTIMIZED
filter_size = trees_client * num_classes # CNN filter size MUST BE equal to the number of trees per client

params_cnn = (num_clients, filter_size, trees_client, filters, num_classes)
models_clients = []  # list of models

model_global = CNN_mc(*params_cnn)  # global model
num_layers = len(model_global.get_weights())

model_global.summary()



Model: "sequential_126"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_126 (Conv1D)         (None, 17, 32)            1312      
                                                                 
 flatten_126 (Flatten)       (None, 544)               0         
                                                                 
 dense_252 (Dense)           (None, 160)               87200     
                                                                 
 dense_253 (Dense)           (None, 4)                 644       
                                                                 
Total params: 89156 (348.27 KB)
Trainable params: 89156 (348.27 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


# Federated Learning process 
Federated Averaging with Adam optimizer simulator. No MQTT.

In [20]:
R = 25  # global FL rounds
E = 10  # local epochs

print(f"Round 0/{R}")  # init model

# sets labels to categorical
categorical_y_valid = np.squeeze(np.eye(num_classes)[y_valid.reshape(-1)])

for r in range(R):  # for each round
    
    # update phase for each client
    for c, (x_train_c, y_train_c) in enumerate(datasets_out):  
        print(f"Round {r + 1}/{R}, Client {c + 1}/{num_clients}")
        model_client = CNN_mc(*params_cnn)  # create a new model
        # set global weights (no memory of prev local weights)
        model_client.set_weights(model_global.get_weights())  
        # update phase
        model_client.fit(
            x_train_c, y_train_c, epochs=E, verbose=False
        )  # train the model on the client data
        models_clients.append(model_client)  # save the model
    
    # aggregation phase
    global_weights = []
    for i in range(num_layers):  # aggregate the weights, no memory of prev global weights
        global_weights.append(
            np.sum([model.get_weights()[i] for model in models_clients], axis=0)
            / len(models_clients)
        )
    model_global.set_weights(global_weights)

    model_global.evaluate(xgb_valid_out, categorical_y_valid)  # evaluate the global model


Round 0/25
Round 1/25, Client 1/5
Round 1/25, Client 2/5
Round 1/25, Client 3/5
Round 1/25, Client 4/5
Round 1/25, Client 5/5
13/13 [==============================] - 0s 2ms/step - loss: 1.1259 - accuracy: 0.7125
Round 2/25, Client 1/5
Round 2/25, Client 2/5
Round 2/25, Client 3/5
Round 2/25, Client 4/5
Round 2/25, Client 5/5
13/13 [==============================] - 0s 2ms/step - loss: 1.2037 - accuracy: 0.7275
Round 3/25, Client 1/5
Round 3/25, Client 2/5
Round 3/25, Client 3/5
Round 3/25, Client 4/5
Round 3/25, Client 5/5
13/13 [==============================] - 0s 3ms/step - loss: 1.2526 - accuracy: 0.7350
Round 4/25, Client 1/5
Round 4/25, Client 2/5
Round 4/25, Client 3/5
Round 4/25, Client 4/5
Round 4/25, Client 5/5
13/13 [==============================] - 0s 2ms/step - loss: 1.2778 - accuracy: 0.7375
Round 5/25, Client 1/5
Round 5/25, Client 2/5
Round 5/25, Client 3/5
Round 5/25, Client 4/5
Round 5/25, Client 5/5
13/13 [==============================] - 0s 2ms/step - loss: 1.272

Final testing 

In [22]:
import scipy.io as sio

y_hat_xgb = model_global.predict(xgb_valid_out)
y_hat_xgb_cont = np.argmax(y_hat_xgb, axis=1)
accuracy_fed = accuracy_score(y_valid, y_hat_xgb_cont)
cm = confusion_matrix(y_valid, y_hat_xgb_cont)

# performance and confusion matrix

print(f"Accuracy (Centralized): {accuracy_s :.2f}")
for c, error in enumerate(errors_clients):
    print(f"Accuracy (Client {c}): {error :.2f}")
print(f"Accuracy (Federated): {accuracy_fed :.2f}")

# saving results
checkpointpath = 'xgb_models/XGB_federated_model_regression_multiclass.h5'
model_global.save(checkpointpath)
# joblib.dump(model_global, checkpointpath, compress=0)
# saving results
# checkpointpath = 'xgb_models/XGB_federated_model.h5'
# model_global.save(checkpointpath)
## joblib.dump(model_global, checkpointpath, compress=0)
# dict_1 = {"Accuracy_centralized": error_centr,
#          "TPR_centralized":  TPR_centralized,
#          "TNR_centralized":  TNR_centralized,
#          "Accuracy_clients": errors_clients,
#          "TPR_clients": TPR_clients,
#          "TNR_clients": TNR_clients,
#          "Accuracy_federation": error_fed,
#          "TPR_federation": TPR_fed,
#          "TNR_federation": TNR_fed,
#
#          }
# sio.savemat(
#    "results/fedXGboost_{}_alpha_{}_samples_{}_run_{}.mat".format('iid',0,100,run), dict_1)


13/13 [==============================] - 0s 3ms/step
Accuracy (Centralized): 0.78
Accuracy (Client 0): 0.58
Accuracy (Client 1): 0.59
Accuracy (Client 2): 0.64
Accuracy (Client 3): 0.61
Accuracy (Client 4): 0.65
Accuracy (Federated): 0.74


c:\Users\admin\anaconda3\envs\xgboost_fed\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
